# Invariant Linear Probing

Дизайн эксперимента:
- скормить трансформеру диалог
- взять скрытые представления со всех слоев
- для скрытого представления предсказать был ли аугментирован диалог или нет
- или не предсказать а спроецировать тсне или пца

## PCA

- взять все пять версий датасета (ориг, инсерт, реплейс, прун, шафл)
- собрать хидены после эмбединг слоя

In [1]:
import numpy as np

sgd_labels = [
    "Banks",
    "Buses",
    "Calendar",
    "Events",
    "Flights",
    "Homes",
    "Hotels",
    "Media",
    "Movies",
    "Music",
    "RentalCars",
    "Restaurants",
    "RideSharing",
    "Services",
]


def add_const_column(dataset, name, value):
    dataset = dataset.add_column(name, [value] * len(dataset))
    return dataset

def get_labels(onehot):
    index = np.nonzero(onehot)[0][0]
    return {"services": sgd_labels[index]}


def oh_to_str(dataset):
    return dataset.map(get_labels, input_columns="services")

In [3]:
from datasets import load_from_disk

ds_path = "DSE-filtered"

In [4]:
aug = 'insert'
dataset_aug = load_from_disk(f"data/scatter-analysis/{ds_path}/sgd-{aug}")
dataset_aug = oh_to_str(dataset_aug)
dataset_aug = add_const_column(dataset_aug, "aug", aug)
dataset_aug

Map:   0%|          | 0/6893 [00:00<?, ? examples/s]

Dataset({
    features: ['content', 'source_dataset_name', 'idx_within_source', 'services', 'embedding', 'hidden_state[-4]', 'hidden_state[-3]', 'hidden_state[-2]', 'hidden_state[-1]', 'aug'],
    num_rows: 6893
})

In [6]:
dataset_aug[0]['hidden_state[-4]']

[-0.2320059984922409,
 0.23996101319789886,
 0.10450970381498337,
 -0.07985961437225342,
 0.46318304538726807,
 0.08456353098154068,
 0.4218515157699585,
 -0.10330381989479065,
 0.040480196475982666,
 -0.25691482424736023,
 0.01578069105744362,
 0.08592332154512405,
 0.22938212752342224,
 0.06331662833690643,
 0.00977612566202879,
 0.2412271797657013,
 0.018951835110783577,
 0.43873801827430725,
 -0.06418764591217041,
 0.3803122639656067,
 0.3487134277820587,
 -0.0222758911550045,
 0.0029457169584929943,
 0.15205617249011993,
 -0.009817780926823616,
 0.38868892192840576,
 0.3280121386051178,
 -0.01249913964420557,
 0.15947818756103516,
 0.12436547130346298,
 0.06562584638595581,
 0.309378057718277,
 0.20962436497211456,
 -0.056193266063928604,
 0.21517443656921387,
 -0.014897891320288181,
 -0.08244330435991287,
 0.05338304117321968,
 -0.1754073202610016,
 -0.14021402597427368,
 -0.07380405813455582,
 -0.13471046090126038,
 0.4718252420425415,
 -0.32595527172088623,
 -0.0442460402846336

In [48]:
aug = 'replace'
dataset_aug2 = load_from_disk(f"data/scatter-analysis/{ds_path}/sgd-{aug}")
dataset_aug2 = oh_to_str(dataset_aug2)
dataset_aug2 = add_const_column(dataset_aug2, "aug", aug)
dataset_aug2

Map:   0%|          | 0/6909 [00:00<?, ? examples/s]

Dataset({
    features: ['content', 'source_dataset_name', 'idx_within_source', 'services', 'embedding', 'aug'],
    num_rows: 6909
})

In [40]:
dataset = load_from_disk(f"data/scatter-analysis/{ds_path}/sgd")
dataset = oh_to_str(dataset)
dataset = add_const_column(dataset, "aug", "orig")

Map:   0%|          | 0/6911 [00:00<?, ? examples/s]

In [41]:
dataset

Dataset({
    features: ['content', 'source_dataset_name', 'idx_within_source', 'services', 'embedding', 'aug'],
    num_rows: 6911
})

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


def aug_prediction(ds_list, seed=0):
    df_list = [ds.to_pandas() for ds in ds_list]

    df_joined = df_list[0].join(df_list[1], on="idx_within_source", how="inner", rsuffix="_1")

    if len(df_list) > 1:
        for i in range(2, len(df_list)):
            df_joined = df_joined.join(df_list[i], on="idx_within_source", how="inner", rsuffix=f"_{i}")
    df_joined.rename(columns={'embedding': 'embedding_0'}, inplace=True)
    
    y = np.random.choice(len(df_list), size=len(df_joined))
    X = [df_joined.loc[i, f'embedding_{aug}'] for i, aug in enumerate(y)]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)

    logreg = LogisticRegression(max_iter=1000)
    logreg.fit(X_train, y_train)
    logreg_accuracy = logreg.score(X_test, y_test)

    knn = KNeighborsClassifier(metric='cosine')
    knn.fit(X_train, y_train)
    knn_accuracy = knn.score(X_test, y_test)
    
    return logreg_accuracy, knn_accuracy

In [52]:
aug_prediction([dataset, dataset_aug, dataset_aug2])

(0.8642659279778393, 0.4445983379501385)